In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from rdkit import Chem
from rdkit.Chem import inchi
from tqdm import tqdm
from time import sleep
from tqdm.notebook import tqdm

from chembl_structure_pipeline import standardizer as ChEMBL_standardizer
from papyrus_structure_pipeline import standardize

[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>]
[ForwardRef('ExtensionArray'), <class 'numpy.ndarray'>, ForwardRef('Index'), ForwardRef('Series')]
[<class 'str'>, <class 'float'>, <class 'bool'>]
[ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta')]
[ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta'), ForwardRef('Interval')]
[<class 'str'>, <class 'float'>, <class 'bool'>, ForwardRef('Period'), ForwardRef('Timestamp'), ForwardRef('Timedelta'), ForwardRef('Interval'), <class 'numpy.datetime64'>, <class 'numpy.timedelta64'>, <class 'datetime.datetime'>]
[ForwardRef('Timestamp'), <class 'datetime.datetime'>, <class 'numpy.datetime64'>, <class 'numpy.int64'>, <class 'float'>, <class 'str'>]
[ForwardRef('Timedelta'), <class 'datetime.timedelta'>, <class 'numpy.timedelta64'>, <class 'numpy.int64'>, <class 'float'>, <class 'str'>]
[<class 'str'>, <class 'datetime.tzinfo'>]
[<class 'str'>, <class 'int'>]
[typing.Hashable, typing.Sequence[typing.Ha

[15:01:52] Initializing Normalizer


In [2]:
#Read the PAMPA data

pampa_train = pd.read_csv('raw_data/pampa/pampa_papp_from_article.csv')

pampa_val_bi = pd.read_csv('raw_data/pampa/papp_pampa_high_low.csv', delimiter = ',')
pampa_val_cont= pd.read_csv('raw_data/pampa/papp_pampa_speed.csv', delimiter=',')

In [3]:
#Select columns we need
pampa_train = pampa_train[['PUBCHEM_EXT_DATASOURCE_SMILES','Phenotype','Permeability']]
pampa_val_bi = pampa_val_bi[['SMILES','Papp(PAMPA)(qualitative)']]
pampa_val_cont = pampa_val_cont[['SMILES','Papp(PAMPA) {measured, converted}']]

In [4]:
#Categorize cont_bbb data as high or low permeable

def compare_values_val(row):
    if row['Papp(PAMPA) {measured, converted}'] > 4:
        return 'high'
    elif row['Papp(PAMPA) {measured, converted}'] < 2:
        return 'low'
    else:
        return '?'

pampa_val_cont['status_pampa'] = pampa_val_cont.apply(compare_values_val, axis=1)


In [5]:
#Categorize training data as high or low permeable

def compare_values_train(row):

    if row['Permeability'] > 4:
        return 'high'
    elif row['Permeability'] < 2:
        return 'low'
    else:
        return '?'

pampa_train['Permeability'] = pampa_train['Permeability'].astype(float)
pampa_train['status_pampa'] = pampa_train.apply(compare_values_train, axis=1)


In [6]:
#Rename columns
pampa_train.rename(columns={'PUBCHEM_EXT_DATASOURCE_SMILES':'SMILES_raw'}, inplace=True)
pampa_val_cont.rename(columns={'SMILES':'SMILES_raw'}, inplace=True)
pampa_val_bi.rename(columns={'SMILES':'SMILES_raw','Papp(PAMPA)(qualitative)':"status_pampa"}, inplace=True)

In [7]:
#Keep only compounds with class
print(f'Full dataset: {len(pampa_train)}')
pampa_train_filt = pampa_train[pampa_train['status_pampa'].isin(['high', 'low'])]
print(f'Compounds with class: {len(pampa_train_filt)}')

Full dataset: 438
Compounds with class: 404


In [8]:
#Remove rows without SMILES

def remove_nan_smiles(df):

    df = df[~(df['SMILES_raw'].isna())]
    df = df.reset_index(drop=True)

    return df

In [9]:
#Papyrus Standardization

def create_sd_smiles(sd_mol):
    try:
        standardized_smiles =  Chem.MolToSmiles(sd_mol)
        return standardized_smiles
    except Exception as e:
        print(f"An sd_smiles error occurred: {str(e)}")
        return None
    
#Create InChI keys from standardized molecules
def mol_to_inchi_key(sd_mol):
    if sd_mol is not None:

        inchi_str = inchi.MolToInchi(sd_mol)
        inchi_key = inchi.InchiToInchiKey(inchi_str)
    else:
        inchi_key = None   
    return inchi_key

def standardize_molecule(mol):
    standardized_mol =  standardize(mol,raise_error=False )
    return standardized_mol

#Standardize 

def standardize_workflow(df_raw):
    for i in range(0,len(df_raw)):
        smiles =df_raw.at[i,'SMILES_raw']
        mol = Chem.MolFromSmiles(smiles)
        sd_mol =  standardize_molecule(mol)
        sd_smiles = create_sd_smiles(sd_mol)
        sd_inchi_key = mol_to_inchi_key(sd_mol)
        df_raw.at[i,'papyrus_SMILES'] = sd_smiles
        df_raw.at[i,'papyrus_inchi_key'] = sd_inchi_key

    print(f'df length after standardization: {len(df_raw)}')

    return df_raw

In [10]:
#Check for missing inchi key

def missing_inchi(df_raw):
    smiles_nan = df_raw['papyrus_SMILES'].isna().sum()
    inchikey_nan =df_raw['papyrus_inchi_key'].isna().sum()
    print(f'DB length: {len(df_raw)},        SMILES nan: {smiles_nan},        inchi key nan: {inchikey_nan}')

    #Remove rows with missing inchikey
    df_valid_inchi= df_raw[((df_raw['papyrus_inchi_key'].notna()))]
    print('-----remove missing inchikey----')
    print(f'updated length: {len(df_valid_inchi)}')

    return df_valid_inchi

In [11]:
#Create connectivity inchi

def inchi_first_part(inchi):
    return inchi.split('-')[0]

def create_connectivity_inchi(df):
    df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
    
    return df

In [12]:
#Check for duplicates

def remove_duplicates(df):
    print(f'length: {len(df)}')
    inchi_un = df['inchi_connectivity'].nunique()
    print(f'unique_inchi: {inchi_un}')


    unique_counts = df.groupby('inchi_connectivity')['status_pampa'].nunique()
    duplicates_diff_class = unique_counts[unique_counts > 1].index

    print(f'Contradicting duplicates: {len(duplicates_diff_class)}')

    #Remove duplicates
    df = df[~(df['inchi_connectivity'].isin(duplicates_diff_class))]
    print(f'After contradicting removed: {len(df)}')
    print(df['inchi_connectivity'].nunique())

    df=df.drop_duplicates(subset=['inchi_connectivity'], keep="first").reset_index(drop=True)
    print(f'After duplicated removed: {len(df)}')
    #df['inchi_stereo'].values_counts

    return df

In [13]:
def class_code(df):
    df['status_pampa'] = df['status_pampa'].replace({'high': 1, 'low': 0})

    return df

In [14]:
#Prepare training data

df = pampa_train_filt

df = remove_nan_smiles(df)

df_sd = standardize_workflow(df)
df_valid = missing_inchi(df_sd)
df_connectivity_inchi = create_connectivity_inchi(df_valid)
df_no_duplicates = remove_duplicates(df_connectivity_inchi)
df_class_coded = class_code(df_no_duplicates)

df_class_coded.to_csv('train_data/kadar_pampa_train.csv')


[15:06:48] WARNING: Omitted undefined stereo

[15:06:48] WARNING: Omitted undefined stereo

[15:06:48] WARNING: Omitted undefined stereo

[15:06:48] WARNING: Proton(s) added/removed

[15:06:48] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:49] WARNING: Omitted undefined stereo

[15:06:50] WARNING: Omitted undefined stereo

[15:06:50] WARNING: Omitted undefined stereo

[15:06:50] WARNING: Omitted undefined stereo

[15:06:50] WARNING: Omitted undefined stereo

[15:06:50] WARNING: Omitted undefined stereo

[15:06:50] WARNING: Omitted undefin

An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[15:06:55] WARNING: Omitted undefined stereo

[15:06:55] WARNING: Omitted undefined stereo

[15:06:56] WARNING: Omitted undefined stereo

[15:06:56] WARNING: Omitted undefined stereo

[15:06:56] WARNING: Omitted undefined stereo

[15:06:56] WARNING: Omitted undefined stereo

[15:06:56] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[15:06:57] WARNING: Omitted undefined stereo

[15:06:57] WARNING: Omitted undefined stereo



...

[15:07:03] WARNING: Charges were rearranged

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:03] WARNING: Omitted undefined stereo

[15:07:04] WARNING: Omitted undefined stereo

[15:07:04] WARNING: Omitted undefined stereo

[15:07:05] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[15:07:06] WARNING: Omitted undefined stereo

[15:07:06] WARNING: Charges were rearranged



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
df length after standardization: 404
DB length: 404,        SMILES nan: 15,        inchi key nan: 15
-----remove missing inchikey----
updated length: 389
length

[15:07:06] WARNING: Omitted undefined stereo

C:\Users\Lily\AppData\Local\Temp\ipykernel_15512\3650569897.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)


In [15]:
#Prepare binary validation data

pampa_val_bi = remove_nan_smiles(pampa_val_bi)

pampa_val_bi_sd = standardize_workflow(pampa_val_bi)
pampa_val_bi_valid = missing_inchi(pampa_val_bi_sd)
pampa_val_bi_connectivity_inchi = create_connectivity_inchi(pampa_val_bi_valid)
pampa_val_bi_no_duplicates = remove_duplicates(pampa_val_bi_connectivity_inchi)
pampa_val_bi_class_coded = class_code(pampa_val_bi_no_duplicates)


An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo

[15:07:12] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


...

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[15:07:40] WARNING: Proton(s) added/removed; Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Charges were rearranged

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo

[15:07:40] WARNING: Omitted undefined stereo



df length after standardization: 1428
DB length: 1428,        SMILES nan: 174,        inchi key nan: 174
-----remove missing inchikey----
updated length: 1254
length: 1254
unique_inchi: 208
Contradicting duplicates: 117
After contradicting removed: 552
91
After duplicated removed: 91


[15:07:40] WARNING: Omitted undefined stereo

[15:07:41] WARNING: Omitted undefined stereo

[15:07:41] WARNING: Omitted undefined stereo

[15:07:41] WARNING: Omitted undefined stereo

C:\Users\Lily\AppData\Local\Temp\ipykernel_15512\3650569897.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)


In [16]:
#Prepare continous validation data

pampa_val_cont = remove_nan_smiles(pampa_val_cont)

pampa_val_cont_sd = standardize_workflow(pampa_val_cont)
pampa_val_cont_valid = missing_inchi(pampa_val_cont_sd)
pampa_val_cont_connectivity_inchi = create_connectivity_inchi(pampa_val_cont_valid)
pampa_val_cont_no_duplicates = remove_duplicates(pampa_val_cont_connectivity_inchi)
filtered_cont_pampa = pampa_val_cont_no_duplicates[~(pampa_val_cont_no_duplicates['status_pampa'] == '?')]

pampa_val_cont_class_coded = class_code(filtered_cont_pampa)



[15:08:53] WARNING: Omitted undefined stereo

[15:08:53] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[15:08:53] WARNING: Omitted undefined stereo

[15:08:53] WARNING: Omitted undefined stereo

[15:08:53] WARNING: Omitted undefined stereo

[15:08:53] WARNING: Omitted undefined stereo

[15:08:53] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::

[15:08:54] WARNING: Omitted undefined stereo

[15:08:54] WARNING: Omitted undefined stereo

[15:08:54] WARNING: Omitted undefined stereo

[15:08:54] WARNING: Charges were rearranged; Omitted undefined stereo

[15:08:54] WARNING: Omitted undefined stereo

[15:08:54] WARNING: Omitted undefined stereo

[15:08:54] WARNING: Omitted undefined stereo



...

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Charges were rearranged; Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)


[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo

[15:08:56] WARNING: Omitted undefined stereo



An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
An sd_smiles error occurred: Python argument types in
    rdkit.Chem.rdmolfiles.MolToSmiles(NoneType)
did not match C++ signature:
    MolToSmiles(class RDKit::ROMol mol, bool isomericSmiles=True, bool kekuleSmiles=False, int rootedAtAtom=-1, bool canonical=True, bool allBondsExplicit=False, bool allHsExplicit=False, bool doRandom=False)
    MolToSmiles(class RDKit::ROMol mol, struct RDKit::SmilesWriteParams params)
df length after standardization: 136
DB length: 136,        SMILES nan: 18,        inchi key nan: 18
-----remove missing inchikey----
updated length: 118
length

[15:08:56] WARNING: Omitted undefined stereo

[15:08:57] WARNING: Omitted undefined stereo

[15:08:57] WARNING: Charges were rearranged

[15:08:57] WARNING: Omitted undefined stereo

[15:08:57] WARNING: Omitted undefined stereo

[15:08:57] WARNING: Omitted undefined stereo

[15:08:57] WARNING: Omitted undefined stereo

[15:08:57] WARNING: Omitted undefined stereo

C:\Users\Lily\AppData\Local\Temp\ipykernel_15512\3650569897.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inchi_connectivity'] = df['papyrus_inchi_key'].apply(inchi_first_part)
C:\Users\Lily\AppData\Local\Temp\ipykernel_15512\1957163213.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

Combining validation sets

In [17]:
df_combined_val = pd.concat([pampa_val_bi_class_coded,pampa_val_cont_class_coded], ignore_index=True, join="inner")

In [20]:
df_val_no_dup = remove_duplicates(df_combined_val)
df_val_no_dup.to_csv('val_data/kadar_pampa_val.csv', index=True)

length: 153
unique_inchi: 140
Contradicting duplicates: 5
After contradicting removed: 143
135
After duplicated removed: 135
